# 简体中文版本

由于中国大陆同学不能正常访问 OpenAI，因此在原版英文教程的基础上，使用 DeepSeek API 完成任务。

注意: 

本材料只是补充，依旧推荐根据官方英语教程完成所有的 coding 练习，只将本材料作为辅助参考。

## 测试 DeepSeek API 可用性

要求:

- `llm_engineering` 目录下已经添加了 `.env` 文件

- `.env` 文件中设置的 `OPENAI_API_BASE` 与 `OPENAI_API_KEY` 为 DeepSeek 对应的

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')

print(f'api_key: {api_key}')
print(f'base_url: {base_url}')

In [ ]:
openai = OpenAI(api_key=api_key, base_url=base_url)
message = "Hello, DeepSeek! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(
    model="deepseek-chat", 
    messages=[
        {"role":"user", "content":message}
    ])
print(response.choices[0].message.content)

## AI 总结网页

### 1. 获取网页内容

我们这里使用 BeautifulSoup 库获取网页内容，并丢弃了 JavaScript, 样式, 图片，输入框等网页内容，对于一些传统的静态网站效果还不错; 

但如果是一些使用大量 JavaScript 的动态网站，例如 OpenAI 官网，就不行了。

可以使用 selenium 库解决该问题。参考 `week1/community-contributions/day1-selenium-for-javascript-sites.ipynb`

In [28]:
# 使用 类 来创建一个网站对象，这个对象包含了网站的标题和文本内容

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# 获取教程作者 Ed 的个人网站

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

### 2. 设置提示词

分为系统提示词和用户提示词

In [1]:
system_prompt = "你是一个助手，负责分析网站的内容，并提供一个简短的摘要，忽略可能与导航相关的文本。\
请使用 Markdown 格式回复。"

In [2]:
def user_prompt_for(website):
    user_prompt = f"你正在查看一个名为 {website.title} 的网站"
    user_prompt += "\n该网站的内容如下；请用 Markdown 格式提供该网站的简短摘要。\
如果它包含新闻或公告，也请对其进行总结。\n\n"
    user_prompt += website.text
    return user_prompt


In [ ]:
print(user_prompt_for(ed))

### 3. 构建发送给 AI 的消息

In [33]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(ed)

### 4. 汇总上面的组件

In [37]:
def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = openai.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://edwarddonner.com")

In [39]:
# 使用 Markdown 显示总结
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://edwarddonner.com")

## 练习

In [49]:
# 三对双引号 ( """ ... """ ) 定义多行字符串
system_prompt = """你是一位尖锐、犀利、有时甚至是刻薄的商业评论家。你的任务是对知名公司进行评论，重点关注它们的战略、产品、公关、以及其他任何可能影响其声誉和盈利能力的关键方面。 你擅长发现问题，并用引人入胜、充满讽刺和幽默的语言表达你的观点。 
"""

user_prompt = """请评论苹果公司最近发布的 Vision Pro 头显。"""

In [50]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
response = openai.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)

print(response.choices[0].message.content)